# Sentiment Analysis of Twitter Posts
<!-- Notebook name goes here -->
<center><b>Notebook: Data Description, Cleaning, Exploratory Data Analysis, and Preprocessing</b></center>
<br>

**By**: Stephen Borja, Justin Ching, Erin Chua, and Zhean Ganituen.

**Dataset**: Hussein, S. (2021). Twitter Sentiments Dataset [Dataset]. Mendeley. https://doi.org/10.17632/Z9ZW7NT5H2.1

**Motivation**: Every minute, social media users generate a large influx of textual data on live events. Performing sentiment analysis on this data provides a real-time view of public perception, enabling quick insights into the general population’s opinions and reactions.

**Goal**: By the end of the project, our goal is to create and compare supervised learning algorithms for sentiment analysis.

### **Dataset Description**

The Twitter Sentiments Dataset is a dataset that contains nearly 163k tweets from Twitter. The time period of when these were collected is unknown, but it was published to Mendeley Data on May 14, 2021 by Sherif Hussein of Mansoura University.

Tweets were extracted using the Twitter API, but the specifics of how the tweets were selected are unmentioned. The tweets are mostly English with a mix of some Hindi words for code-switching<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). All of them seem to be talking about the political state of India. Most tweets mention Narendra Modi, the current Prime Minister of India.

Each tweet was assigned a label using TextBlob's sentiment analysis (El‑Demerdash, Hussein, & Zaki, 2021), which assigns labels automatically.

Twitter_Data
- **`clean_text`**: The tweet's text
- **`category`**: The tweet's sentiment category

What each row and column represents: `each row represents one tweet.` <br>
Number of observations: `162,980`

---
**References**
1. Alaa A. El-Demerdash, J. F. W. Z., Sherif E. Hussein. (2022). Course Evaluation Based on Deep Learning and SSA Hyperparameters Optimization. Computers, Materials & Continua, 71(1), 941–959. doi:10.32604/cmc.2022.021839

---

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Code-switching is the practice of alternating between two languages $L_1$ (the native language) and $L_2$ (the source language) in a conversation. In this context, the code-switching is done to appear more casual since the conversation is done via Twitter (now, X). 

## 1 project set up
Set up here the imports for the projects (ensure these are installed via uv and is part of the environment). Furthermore, load the dataset here. Also load the raw dataset here.

In [33]:
import pandas as pd
import numpy as np

# Import boilerplate file generated by `codegen.py`
from boilerplate import stopwords_set

# Cleaning Imports
import unicodedata
import re
import string

# Imports for Stemming and Lematization
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Imports for Text Vectorization
from sklearn.feature_extraction.text import CountVectorizer

# Set up NLTK objects for stemming and lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Set up scikit-learn objects for text vectorization
vectorizer = CountVectorizer()

# Load raw data file
df = pd.read_csv("../data/Twitter_Data.csv")
df = df.dropna()

[nltk_data] Downloading package wordnet to /home/zrgnt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zrgnt/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## 2 data cleaning
This section discusses the methodology for data cleaning.

We follow a similar methodology for data cleaning presented in [1].

The cleaning pipeline has four main functions. The first function is the `normalize` function, it normalizes the text input to ASCII-only characters (say, "cómo estás" becomes "como estas") and lowercases alphabetic symbols. The dataset contains Unicode characters (e.g., emojis and accented characters) which the function replaces to the empty string (`''`). Then, `rem_punctuation` removes the punctuation marks and special characters with the empty string. Then, `collapse_whitespace` collapses all whitespace characters to a single space. Formally, it is a transducer from $\Box^+ \mapsto \Box$ where $\Box$ is the whitespace character. Now, since the strings are cleaned at this point, the tokenization step reduces to a mere string split at word boundaries. Finally, with the tokenized string, we can do a final clean by removing the stopwords.

For stop words removal, we refer to the English stopwords dataset defined in NLTK and Wolfram Mathematica [2,3].

---
**References**
1. George, M., & Murugesan, R., Dr. (2024). Improving sentiment analysis of financial news headlines using hybrid Word2Vec-TFIDF feature extraction technique. Procedia Computer Science, 244, 1–8. https://doi.org/10.1016/j.procs.2024.10.172
2. Wolfram Language. (2015). DeleteStopwords. Wolfram Language & System Documentation Center. Retrieved from https://reference.wolfram.com/language/ref/DeleteStopwords.html
3. Bird, S., & Loper, E. (2004, July). NLTK: The Natural Language Toolkit. Proceedings of the ACL Interactive Poster and Demonstration Sessions, 214–217. Retrieved from https://aclanthology.org/P04-3031/

In [35]:
def normalize(text: str) -> str:
    """
    Normalize text from a pandas entry to ASCII-only lowercase characters. Hence, this removes Unicode characters with no ASCII
    equivalent (e.g., emojis and CJKs).

    Do not use this function alone, use `clean_and_tokenize()`.
    
    # Parameters
    * text: String entry.

    # Returns
    ASCII-normalized text containing only lowercase letters.

    # Examples
    normalize("¿Cómo estás?")
    $ 'como estas?'

    normalize(" hahahaha HUY! Kamusta 😅 Mayaman $$$ ka na ba?") 
    $ ' hahahaha huy! kamusta  mayaman $$$ ka na ba?'
    """
    normalized = unicodedata.normalize('NFKD', text)
    ascii_text = normalized.encode('ascii', 'ignore').decode('ascii')

    return ascii_text.lower()


def rem_punctuation(text: str) -> str:
    """
    Removes the punctuations. This function simply replaces all punctuation marks and special characters
    to the empty string. Hence, for symbols enclosed by whitespace, the whitespace are not collapsed to a single whitespace
    (for more information, see the examples).

    Do not use this function alone, use `clean_and_tokenize()`.
    
    # Parameters
    * text: String entry.

    # Returns
    Text with the punctuation removed, or None if the result is empty or input invalid.

    # Examples
    rem_punctuation("this word $$ has two spaces after it!")
    $ 'this word  has two spaces after it'

    rem_punctuation("these!words@have$no%space")
    $ 'thesewordshavenospace'
    """
    return re.sub(f"[{re.escape(string.punctuation)}]", "", text)


def collapse_whitespace(text: str) -> str:
    """
    This collapses whitespace. Here, collapsing means the transduction of all whitespace strings of any
    length to a whitespace string of unit length (e.g., "   " -> " "; formally " "+ -> " ").
    
    Do not use this function alone, use `clean_and_tokenize()`.

    # Parameters
    * text: String entry.

    # Returns
    Text with the whitespaces collapsed.

    # Examples
    collapse_whitespace("  huh,  was.  that!!! ")
    $ 'huh, was. that!!!'
    """
    return re.sub(f" +", " ", text).strip()


def rem_stopwords(tokens: str, stopwords: set[str]) -> str:
    """
    This removes all stopwords. For fast look up, `stopwords` is a set with type str.
    This allows for constant time lookups as opposed to a linear search with a list.

    Strings detected as stopwords is replaced with the empty string "".

    Do not use this function alone, use `clean_and_tokenize()`.

    # Parameters
    * text: A tokenized string. 
    * stopwords: stopword dictionary.
    
    # Returns
    Text with the stopwords removed.

    # Examples
    rem_stopwords(["he", "is", "an", "amazing", "master",], stopwords_lut)
    $ ['amazing', 'master']

    # Future
    If this function is too slow, we may implement `stopwords` as an 26-ary trie to achieve log-linear time.
    """
    filtered = [word for word in tokens if word not in stopwords]
    return filtered
    
def clean_and_tokenize(text: str, stopwords: set[str]) -> list[str]:
    """
    This is the main function for data cleaning (i.e., it calls all the cleaning functions in the prescribed order).

    This function should be used as a first-class function in a map.

    # Parameters
    * text: The string entry from a DataFrame column.
    * stopwords: stopword dictionary.

    # Returns
    Clean tokenized string. 
    """
    # cleaning on the base string
    text = normalize(text)
    text = rem_punctuation(text)
    text = collapse_whitespace(text)
    
    # tokenization is now trivial since the cleaning steps allow the tokenization to be a mere word boundary split
    toks = text.split()

    # cleaning on the tokenized string
    toks = rem_stopwords(toks, stopwords)

    return toks

# perform cleaning stage
# at this point NaN entries shouldn't exist
df["clean_ours"] = df['clean_text'].map(lambda x: clean_and_tokenize(x, stopwords_set))

# 3 preprocessing

> 🏗️ Perhaps swap S3 and S4. Refer to literature on what comes first.

This section discusses preprocessing steps for the cleaned data.

## Stemming and Lemmatization

We follow a similar methodology for data cleaning presented in [1]. We preprocess the dataset entries via stemming and lemmatization. We employ NLTK for both tasks using PorterStemmer and WordNetLemmatizer for stemming and lemmatization, repectively [2]. For the lemmatization step, we use the WordNet for English lemmatization and Open Multilingual WordNet version 1.4 for translations and multilingual support which is important for our case since some tweets contain text from Indian Languages.

---
**References**
1. George, M., & Murugesan, R., Dr. (2024). Improving sentiment analysis of financial news headlines using hybrid Word2Vec-TFIDF feature extraction technique. Procedia Computer Science, 244, 1–8. https://doi.org/10.1016/j.procs.2024.10.172
2. Bird, S., & Loper, E. (2004, July). NLTK: The Natural Language Toolkit. Proceedings of the ACL Interactive Poster and Demonstration Sessions, 214–217. Retrieved from https://aclanthology.org/P04-3031/

In [36]:
df["stemmed"] = df["clean_ours"].map(lambda tokens: [stemmer.stem(t) for t in tokens])
df["lemmatized"] = df["clean_ours"].map(lambda tokens: [lemmatizer.lemmatize(t) for t in tokens])

# do this, since vectorizer expects a string not an array of strings
df["lemmatized_str"] = df["lemmatized"].apply(lambda x: " ".join(x))

## Vector Representation

After to stemming and lemmatization steps, we can now represent each each entry to its vector representation from a Bag of Words (BoW) model. We use scikit-learn's `CountVectorizer` which is an ready-to-use implementation of the BoW model.

**Related Literature and Vectorization Techniques.** Traditional vectorization techniques include BoW and Term Frequency-Inverse Document Frequency (TF-IDF). TF-IDF weights each word based on its frequency in a document and how rare it is across the corpus, reducing the impact of common words. BoW, in contrast, simply counts word occurrences without considering corpus-level frequency. In this project, BoW was chosen because stopwords were already removed during preprocessing, and the dataset is domain-specific [3]. In such datasets, frequent words are often meaningful domain keywords, so scaling them down (as TF-IDF would) could reduce the importance of these key terms in the feature representation.

The resulting vector has 162,969 rows which is the number of entries in the dataset (with `NaN` entries removed), and 101,284 which is the number of unique words in the dataset.

---

**References**
1. Rani, D., Kumar, R., & Chauhan, N. (2022, October). Study and comparision of vectorization techniques used in text classification. In 2022 13th international conference on computing communication and networking technologies (ICCCNT) (pp. 1-6). IEEE.
2. Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., & Duchesnay, Έ. (2011). Scikit-learn: Machine Learning in Python. J. Mach. Learn. Res., 12(null), 2825–2830.

In [40]:
bow = vectorizer.fit_transform(df["lemmatized_str"])
feature_names = vectorizer.get_feature_names_out()

# testing
sentence = "when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples"
sentence = clean_and_tokenize(sentence, stopwords_set)
sentence = " ".join(sentence)
vec = vectorizer.transform([sentence])

nonzero_idx = vec.nonzero()[1]
list(zip(feature_names[nonzero_idx], vec.data))

[('business', np.int64(1)),
 ('difficult', np.int64(1)),
 ('exit', np.int64(1)),
 ('expected', np.int64(1)),
 ('governance', np.int64(1)),
 ('government', np.int64(1)),
 ('job', np.int64(1)),
 ('justice', np.int64(1)),
 ('maximum', np.int64(1)),
 ('minimum', np.int64(1)),
 ('modi', np.int64(1)),
 ('promised', np.int64(1)),
 ('psus', np.int64(1)),
 ('reforming', np.int64(1)),
 ('state', np.int64(2))]

# 4 exploratory data analysis

This section discusses the exploratory data analysis conducted on the dataset after cleaning.

> Notes from Zhean: <br>
> From manual checking via OpenRefine, there are a total of 162972. `df.info()` should have the same result post-processing.
> Furthermore, there should be two columns, `clean_text` (which is a bit of a misnormer since it is still dirty) contains the Tweets (text data). The second column is the `category` which contains the sentiment of the Tweet and is a tribool (1 positive, 0 neutral or indeterminate, and -1 for negative).

---
**References**
1. ()